In [5]:
import os
import requests
import json

In [4]:
channel_id = 'UCsqjHFMB_JYTaEnf_vmTNqg'
api_key = os.environ['YOUTUBE_API_KEY']

# print(api_key)

## Extraction

In [ ]:
def channel_stats(api_key, channel_id):
    channel_stats_url = f"https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2Cstatistics&id={channel_id}&key={api_key}"
    # print(channel_stats_url)
    response = requests.get(channel_stats_url)
    channel_data = json.loads(response.text)
    # print(channel_data)
    return channel_data

In [3]:
def get_channel_videos(api_key, channel_id):
    channel_videos_url = f"https://youtube.googleapis.com/youtube/v3/search?channelId={channel_id}&maxResults=50&order=date&type=video&key={api_key}"
    print(channel_videos_url)

get_channel_videos(api_key, channel_id)